In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.metrics import Recall
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
data = pd.read_csv('../../Raw_Data/data_cleaned.csv')
data.head()

,is_spoiler,clean_reviews
0,1,classic piece unforgettable film making oscar ...
1,1,simply amazing best film shawshank redemption ...
2,1,best story ever told film believe film best st...
3,1,busy dying busy living yes spoiler film emotio...
4,1,great story wondrously told acted heart extrao...


In [3]:
data.shape

(573913, 2)

In [4]:
df = data.loc[:1000].copy()

In [7]:
X = df[['clean_reviews']]
y = df['is_spoiler']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

In [8]:
def convert_sentences(X):
    return [sentence.split(' ') for sentence in X]

X_train = X_train.apply(convert_sentences)
X_test = X_test.apply(convert_sentences)

In [9]:
word_to_id = {}
iter_ = 1
for sentence in X_train['clean_reviews']:
    for word in sentence:
        if word in word_to_id:
            continue
        word_to_id[word] = iter_
        iter_ += 1

In [10]:
print(f'There are {len(word_to_id)} different words in the train sentences')

There are 8618 different words in the train sentences


In [11]:
def tokenize(sentences, word_to_id):
    return [[word_to_id[_] for _ in s if _ in word_to_id] for s in sentences]

X_token_train = tokenize(X_train['clean_reviews'], word_to_id)
X_token_test = tokenize(X_test['clean_reviews'], word_to_id)

In [14]:
X_train_maxlen = pad_sequences(X_token_train, maxlen=150, dtype='float32', padding='post')
X_test_maxlen = pad_sequences(X_token_test, maxlen=150, dtype='float32', padding='post')

In [17]:
recall = Recall(name='recall')

def init_model(vocab_size):
    model = Sequential()
    model.add(layers.Embedding(input_dim=vocab_size+1, output_dim=30, mask_zero=True))
    model.add(layers.GRU(units=128 , recurrent_dropout = 0.5 , dropout = 0.5))
    model.add(layers.Dense(5, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=recall)
    
    return model

In [20]:
model = init_model(len(word_to_id))
es = EarlyStopping(patience=5, restore_best_weights=True)
model.fit(X_train_maxlen, y_train, epochs=10, batch_size=32, validation_split=0.2, callbacks=[es] )

Epoch 1/10
20/20 [==============================] - 5s 253ms/step - loss: 0.3395 - recall: 0.9755 - val_loss: 0.2346 - val_recall: 1.0000
Epoch 2/10
20/20 [==============================] - 5s 240ms/step - loss: 0.1996 - recall: 1.0000 - val_loss: 0.2028 - val_recall: 1.0000
Epoch 3/10
20/20 [==============================] - 5s 272ms/step - loss: 0.1876 - recall: 1.0000 - val_loss: 0.2015 - val_recall: 1.0000
Epoch 4/10
20/20 [==============================] - 4s 202ms/step - loss: 0.1854 - recall: 1.0000 - val_loss: 0.2006 - val_recall: 1.0000
Epoch 5/10
20/20 [==============================] - 4s 207ms/step - loss: 0.1782 - recall: 1.0000 - val_loss: 0.2127 - val_recall: 1.0000
Epoch 6/10
20/20 [==============================] - 3s 175ms/step - loss: 0.1770 - recall: 1.0000 - val_loss: 0.2046 - val_recall: 1.0000
Epoch 7/10
20/20 [==============================] - 3s 167ms/step - loss: 0.1610 - recall: 1.0000 - val_loss: 0.2387 - val_recall: 1.0000
Epoch 8/10
20/20 [================

In [21]:
model.evaluate(X_test_maxlen, y_test)

7/7 [==============================] - 0s 43ms/step - loss: 0.1833 - recall: 1.0000


[0.183307483792305, 1.0]